In [1]:
import fitz
import pysubs2
from bs4 import BeautifulSoup
from sub_PDF_location import get_time_location
from PIL import Image
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip
from moviepy.editor import ImageClip, concatenate_videoclips


In [2]:
def draw_line(pdf_file_path, page_num, top, left, length,height,color=(1,1,0), dpi=300):
    pdf_document = fitz.open(pdf_file_path)
    page=pdf_document[page_num]
    # 绘制从左下，到右下的直线
    x0=left
    y0=top+height
    x1=left+length
    y1=top+height

    shape=page.new_shape()
    shape.draw_line((x0,y0),(x1,y1)) 
    shape.finish(color=color)
    shape.commit()
    pix = page.get_pixmap(matrix=fitz.Matrix(dpi/72, dpi/72))
    return pix 


def pixmap_to_cv_image(pixmap):
    # 获取图像数据
    image_data = np.frombuffer(pixmap.samples, dtype=np.uint8)
    
    # 将图像数据重新调整为正确的形状
    if pixmap.alpha:
        shape = (pixmap.height, pixmap.width, 4)
    else:
        shape = (pixmap.height, pixmap.width, 3)
    image_data = image_data.reshape(shape)

    # 将图像从RGBA转换为BGR格式
    if pixmap.alpha:
        image_data = cv2.cvtColor(image_data, cv2.COLOR_RGBA2BGR)
    else:
        image_data = cv2.cvtColor(image_data, cv2.COLOR_RGB2BGR)

    return image_data
    

In [3]:
if __name__ == '__main__':
    pdf_file_path = 'test_pdf/test2.pdf'
    vtt_file_path = 'test_pdf/test.srt'
    start_page = 1
    end_page = 1
    time_location = get_time_location(pdf_file_path, vtt_file_path, start_page, end_page,n=0)
    images=[]
    timestamps=[]
    for i, t in enumerate(time_location[:20]):
        # print(t)
        pix=draw_line(pdf_file_path, start_page+t['page_num'], t['top'], t['left'], t['line_length']/2*0.9,t['line_height'])
        # pix.save(f"test_pdf/test{i:03d}.png")
        cv2_img=pixmap_to_cv_image(pix)
        images.append(cv2_img)
        timestamps.append((t['start_time']/1000,t['end_time']/1000))
    


In [4]:
import imageio
# import imageio_ffmpeg as ffmpeg
# import numpy as np
import subprocess 

def create_video_with_audio(images, timestamps, audio_path, output_video, frame_rate):
    # 获取图像尺寸
    height, width, layers = images[0].shape

    # 创建视频编码器
    output_params = ['-r', str(frame_rate), '-c:v', 'libx264', '-preset', 'medium', '-profile:v', 'main', '-pix_fmt', 'yuv420p']
    writer = imageio.get_writer(output_video, format='FFMPEG', mode='I', fps=frame_rate, codec='libx264', output_params=output_params)

    # 将图像添加到视频中
    for start_time, end_time in timestamps:
        duration = end_time - start_time
        total_frames = int(duration * frame_rate)

        for img in images:
            for _ in range(total_frames):
                writer.append_data(img)

    # 关闭视频编码器
    writer.close()

    # 将音频添加到生成的视频中
    output_video_with_audio = 'output_video_with_audio.mp4'
    command = f'ffmpeg -i {output_video} -i {audio_path} -c:v copy -c:a aac -strict experimental -map 0:v:0 -map 1:a:0 {output_video_with_audio}'
    subprocess.call(command, shell=True)

In [5]:
output_video = 'output_video_no_audio.mp4' # 输出无音频的视频文件名
final_video = 'output_video_with_audio.mp4' # 输出带音频的视频文件名
frame_rate = 30 # 视频的帧率
audio_path = 'test_pdf/test.mp3' # 音频文件路径

create_video_with_audio(images, timestamps, audio_path, output_video, frame_rate)
print("done")


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2481, 3260) to (2496, 3264) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


AttributeError: module 'imageio_ffmpeg' has no attribute 'input'